In [0]:
import tensorflow as tf
from google.colab import drive
from IPython.display import Image, display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys, getopt
import json
import cv2
import glob
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 11.5 GB  | Proc size: 142.9 MB
GPU RAM Free: 2974MB | Used: 8467MB | Util  74% | Total 11441MB


In [0]:
printm()

Gen RAM Free: 11.3 GB  | Proc size: 321.7 MB
GPU RAM Free: 2974MB | Used: 8467MB | Util  74% | Total 11441MB


In [0]:
1e-4

0.0001

In [0]:
with open('/content/gdrive/My Drive/yolo.weights', 'rb') as f:
  content = f.read()

In [0]:
content[0]

NameError: ignored

In [0]:
with open('/content/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/gdrive/My\ Drive/foo.txt

In [0]:
img = Image('/content/gdrive/My Drive/cocostuff2017/val2017/000000000139.jpg')

display(img)

In [0]:
def image_example(image_string, label):
  image_shape = tf.image.decode_jpeg(image_string).shape

  feature = {
      'height': _int64_feature(image_shape[0]),
      'width': _int64_feature(image_shape[1]),
      'depth': _int64_feature(image_shape[2]),
      'label': _int64_feature(label),
      'image_raw': _bytes_feature(image_string),
  }

  return tf.train.Example(features=tf.train.Features(feature=feature))

In [0]:
net_scale = 32
grid_w, grid_h = 19, 19
n_classes = 80
iou_th = 0.7
in_w, in_h = grid_w*net_scale, grid_h*net_scale

In [0]:
with open('/content/gdrive/My Drive/cocostuff2017/annotations/instances_val2017.json') as json_data:
    data = json.load(json_data)
    
rois = []
classes = []
for i in data['images']:
  img = cv2.imread('/content/gdrive/My Drive/cocostuff2017/val2017/'+i['file_name'])
    
  for a in data['annotations']:
    if a['image_id']==i['id']:
      rois.append(a['bbox'])
      classes.append(a['category_id'])
      print (a['category_id'], a['bbox'])
  break

44 [217.62, 240.54, 38.99, 57.75]
67 [1.0, 240.24, 346.63, 186.76]
1 [388.66, 69.92, 109.41, 277.62]
49 [135.57, 249.43, 22.32, 28.79]
51 [31.28, 344.0, 68.12, 40.83]
51 [59.63, 287.36, 76.07, 41.3]
79 [1.36, 164.33, 192.56, 98.37]
1 [0.0, 262.81, 62.16, 36.77]
47 [119.4, 272.51, 24.82, 34.25]
47 [141.47, 267.91, 32.19, 35.86]
51 [155.97, 168.95, 26.03, 17.13]
51 [157.2, 114.15, 17.86, 15.82]
56 [98.75, 304.78, 10.78, 5.57]
50 [166.03, 256.36, 8.82, 18.58]
56 [86.41, 293.97, 23.96, 11.18]
56 [70.14, 296.16, 9.28, 4.58]
79 [0.0, 210.9, 191.36, 98.98]
57 [96.69, 297.09, 7.84, 4.86]
81 [497.25, 203.4, 122.01, 28.61]


In [0]:
def get_data():
  with open('/content/gdrive/My Drive/cocostuff2017/annotations/instances_val2017.json') as json_data:
      data = json.load(json_data)
      
  ROI = []
  

  for i in data['images']:
    img = cv2.imread('/content/gdrive/My Drive/cocostuff2017/val2017/'+i['file_name'])
    rois = []
    classes = []
    for a in data['annotations']:
      if a['image_id']==i['id']:
        rois.append(a['bbox'])
        classes.append(a['category_id'])

In [0]:
np.array(rois, dtype=np.float32)

In [0]:
img = cv2.resize(img, (in_w, in_h))

In [0]:
img.tostring()

In [0]:
np.array(image_string)

In [0]:
def read_anchors_file(file_path):

	anchors = []
	with open(file_path, 'r') as file:
		for line in file.read().splitlines():
			anchors.append(map(float,line.split()))

	return np.array(anchors)

def iou_wh(r1, r2):

	min_w = min(r1[0],r2[0])
	min_h = min(r1[1],r2[1])
	area_r1 = r1[0]*r1[1]
	area_r2 = r2[0]*r2[1]
		
	intersect = min_w * min_h		
	union = area_r1 + area_r2 - intersect

	return intersect/union
	
def get_grid_cell(roi, raw_w, raw_h, grid_w, grid_h):

	x_center = roi[0] + roi[2]/2.0
	y_center = roi[1] + roi[3]/2.0

	grid_x = int(x_center/float(raw_w)*float(grid_w))
	grid_y = int(y_center/float(raw_h)*float(grid_h))
		
	return grid_x, grid_y

def get_active_anchors(roi, anchors):
	 
	indxs = []
	iou_max, index_max = 0, 0
	for i,a in enumerate(anchors):
		iou = iou_wh(roi[2:], a)
		if iou>iou_th:
			indxs.append(i)
		if iou > iou_max:
			iou_max, index_max = iou, i

	if len(indxs) == 0:
		indxs.append(index_max)

	return indxs

def roi2label(roi, anchor, raw_w, raw_h, grid_w, grid_h):
	
	x_center = roi[0]+roi[2]/2.0
	y_center = roi[1]+roi[3]/2.0

	grid_x = x_center/float(raw_w)*float(grid_w)
	grid_y = y_center/float(raw_h)*float(grid_h)
	
	grid_x_offset = grid_x - int(grid_x)
	grid_y_offset = grid_y - int(grid_y)

	roi_w_scale = roi[2]/anchor[0]
	roi_h_scale = roi[3]/anchor[1]

	label=[grid_x_offset, grid_y_offset, roi_w_scale, roi_h_scale]
	
	return label

def onehot(idx, num):
	
	ret = np.zeros([num], dtype=np.float32)
	ret[idx] = 1.0
	
	return ret

def read_csv_file(filename):

	filenames = []
	rois = []
	classes = []
	with open(filename) as csvfile:
		i=['filename', 'rois', 'classes']
		csvdata = csv.DictReader(csvfile)
		for row in csvdata:
			filenames.append(row['filename'])
			rois.append(row['rois'])
			classes.append(row['classes'])

	return filenames, rois, classes


In [0]:
n_classes = 80
iou_th = 0.7

In [0]:
def make_tfrecord():
#   anchors = read_anchors_file('/content/gdrive/My Drive/Colab Notebooks/anchors.txt')
  anchors = [[0.57273,0.677385],[1.87446,2.06253],[3.33843,5.47434],[7.88282,3.52778],[9.77052,9.16828]]
  n_anchors = np.shape(anchors)[0]
  print (n_anchors)
  
  
  with open('/content/gdrive/My Drive/cocostuff2017/annotations/instances_val2017.json') as json_data:
    data = json.load(json_data)
  
#   with tf.python_io.TFRecordWriter('/content/gdrive/My Drive/cocostuff2017/val2017.tfrecord') as writer:
  with tf.python_io.TFRecordWriter('./val2017_new.tfrecord') as writer:

    for i in data['images']:
      img = cv2.imread('/content/gdrive/My Drive/cocostuff2017/val2017/'+i['file_name'])
#       print(i['file_name'])
      raw_h = np.shape(img)[0]
      raw_w = np.shape(img)[1]
#       grid_h = raw_h//32
#       grid_w = raw_w//32
#       in_w, in_h = grid_w*32, grid_h*32
      img = cv2.resize(img, (in_w, in_h))
      
      rois = []
      classes = []
      for a in data['annotations']:
        if a['image_id']==i['id']:
          rois.append(a['bbox'])
          classes.append(a['category_id'])
          data['annotations'].remove(a)
          
      rois = np.array(rois, dtype=np.float32)
      classes = np.array(classes, dtype=np.int32)
      
      label = np.zeros([grid_h, grid_w, n_anchors, n_classes], dtype=np.float32)
      
      for roi, cls in zip(rois,classes):
        
        active_indxs = get_active_anchors(roi, anchors)
        grid_x, grid_y = get_grid_cell(roi, raw_w, raw_h, grid_w, grid_h)
        
        for active_indx in active_indxs:
          
          anchor_label = roi2label(roi, anchors[active_indx], raw_w, raw_h, grid_w, grid_h)
          print (cls)
          label[grid_y, grid_x, active_indx] = np.concatenate((anchor_label, [cls], [1.0]))
      image_raw = img.tostring()
      label_raw = label.tostring()
      
      example = tf.train.Example(features=tf.train.Features(feature={
					'label': tf.train.Feature(bytes_list=tf.train.BytesList(value=[label_raw])),
					'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_raw]))}))
      
      writer.write(example.SerializeToString())

In [0]:
make_tfrecord()

5
44


ValueError: ignored

In [0]:
!mv ../val2017.tfrecord .

In [0]:
!pwd

/content


In [0]:
!cp -r ./train2017 gdrive/My\ Drive/cocostuff2017/train2017/

In [0]:
!wget http://images.cocodataset.org/zips/train2017.zip

--2019-03-16 19:42:04--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.105.67
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.105.67|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘train2017.zip’

train2017.zip       100%[===================>]  18.01G  52.7MB/s    in 5m 16s  

2019-03-16 19:47:19 (58.4 MB/s) - ‘train2017.zip’ saved [19336861798/19336861798]



In [0]:
!rm train2017.*

In [0]:
%%capture
!unzip train2017.zip -d .

In [0]:


with tf.python_io.TFRecordWriter('images.tfrecords') as writer:

  for filepath in glob.iglob('/content/gdrive/My Drive/cocostuff2017/val2017/*.jpg'):
      image_string = open(filepath,'rb').read()
      tf_example = image_example(image_string)
      writer.write(tf_example.SerializeToString())

In [0]:
raw_image_dataset = tf.data.TFRecordDataset(['images.tfrecords'])
image_feature_description = {
    'image_raw': tf.FixedLenFeature([], tf.string),
}
def _parse_image_function(example_proto):
  # Parse the input tf.Example proto using the dictionary above.
  return tf.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
parsed_image_dataset

In [0]:
record_iterator = tf.python_io.tf_record_iterator(path='images.tfrecords')

for string_record in record_iterator:
  example = tf.train.Example()
  example.ParseFromString(string_record)
  
  print(example)
  
  # Exit after 1 iteration as this is purely demonstrative.
  break

In [0]:
for image_features in parsed_image_dataset:
  image_raw = image_features['image_raw'].numpy()
  display.display(display.Image(data=image_raw))
  break

In [0]:
SCALE = 32
GRID_W, GRID_H = 18, 10
N_CLASSES = 6
N_ANCHORS = 4
IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH = GRID_H*SCALE, GRID_W*SCALE, 3

In [0]:
reader = tf.TFRecordReader()
filename_queue = tf.train.string_input_producer(['images.tfrecord'], num_epochs=None)
_, serialized_example = reader.read(filename_queue)

min_queue_examples = 500

batch = tf.train.shuffle_batch([serialized_example], batch_size=1, capacity=min_queue_examples+100, min_after_dequeue=min_queue_examples, num_threads=2)
parsed_example = tf.parse_example(batch,features={'image_raw': tf.FixedLenFeature([], tf.string)})

image_raw = tf.decode_raw(parsed_example['image_raw'], tf.uint8)
image = tf.cast(tf.reshape(image_raw, [1, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH]), tf.float32)
image = image/255.0

In [0]:
image

In [0]:
with tf.Session() as sess:
    # display encoded back to image data
    jpeg_bin = sess.run(image)

In [0]:
train_rec = tf.train.Example(features = tf.train.Features(feature=feature))

In [0]:
image_shape = tf.image.decode_jpeg(image_string).shape
print (image_shape[0])

In [0]:
ann = pd.read_json('/content/gdrive/My Drive/cocostuff2017/annotations/instances_val2017.json',orient='records')

In [0]:
import json

In [3]:
with open('/content/gdrive/My Drive/cocostuff2017/annotations/instances_val2017.json','r') as COCO:
  js = json.loads(COCO.read())
  print (json.dumps(js['categories']))

[{"supercategory": "person", "id": 1, "name": "person"}, {"supercategory": "vehicle", "id": 2, "name": "bicycle"}, {"supercategory": "vehicle", "id": 3, "name": "car"}, {"supercategory": "vehicle", "id": 4, "name": "motorcycle"}, {"supercategory": "vehicle", "id": 5, "name": "airplane"}, {"supercategory": "vehicle", "id": 6, "name": "bus"}, {"supercategory": "vehicle", "id": 7, "name": "train"}, {"supercategory": "vehicle", "id": 8, "name": "truck"}, {"supercategory": "vehicle", "id": 9, "name": "boat"}, {"supercategory": "outdoor", "id": 10, "name": "traffic light"}, {"supercategory": "outdoor", "id": 11, "name": "fire hydrant"}, {"supercategory": "outdoor", "id": 13, "name": "stop sign"}, {"supercategory": "outdoor", "id": 14, "name": "parking meter"}, {"supercategory": "outdoor", "id": 15, "name": "bench"}, {"supercategory": "animal", "id": 16, "name": "bird"}, {"supercategory": "animal", "id": 17, "name": "cat"}, {"supercategory": "animal", "id": 18, "name": "dog"}, {"supercategory

In [17]:
lookup_dict = {}
for i,j in enumerate(js['categories']):
  lookup_dict.update({j['id']:i})
  print (j)

{'supercategory': 'person', 'id': 1, 'name': 'person'}
{'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'}
{'supercategory': 'vehicle', 'id': 3, 'name': 'car'}
{'supercategory': 'vehicle', 'id': 4, 'name': 'motorcycle'}
{'supercategory': 'vehicle', 'id': 5, 'name': 'airplane'}
{'supercategory': 'vehicle', 'id': 6, 'name': 'bus'}
{'supercategory': 'vehicle', 'id': 7, 'name': 'train'}
{'supercategory': 'vehicle', 'id': 8, 'name': 'truck'}
{'supercategory': 'vehicle', 'id': 9, 'name': 'boat'}
{'supercategory': 'outdoor', 'id': 10, 'name': 'traffic light'}
{'supercategory': 'outdoor', 'id': 11, 'name': 'fire hydrant'}
{'supercategory': 'outdoor', 'id': 13, 'name': 'stop sign'}
{'supercategory': 'outdoor', 'id': 14, 'name': 'parking meter'}
{'supercategory': 'outdoor', 'id': 15, 'name': 'bench'}
{'supercategory': 'animal', 'id': 16, 'name': 'bird'}
{'supercategory': 'animal', 'id': 17, 'name': 'cat'}
{'supercategory': 'animal', 'id': 18, 'name': 'dog'}
{'supercategory': 'animal', 'id':

In [9]:
lookup_dict[16]

14

In [0]:
import numpy as np

In [0]:
np.save("/content/gdrive/My Drive/lookup_dict.npy", lookup_dict)

In [0]:
with open('/content/gdrive/My Drive/coco_label.txt', 'w') as f:
  for j,i in enumerate(js['categories']):
    f.write(str(j)+' '+i['name'].replace(' ','_')+' '+'(0,0,255)\n')
 

In [0]:
with open('/content/gdrive/My Drive/cocostuff2017/annotations/instances_val2017.json') as json_data:
    data = json.load(json_data)

# using the from_dict load function. Note that the 'orient' parameter 
#is not using the default value (or it will give the same error than you had)
# We transpose the resulting df and set index column as its index to get this result
# pd.DataFrame.from_dict(data, orient='index').T.set_index('id')   

In [0]:
data.keys()

In [0]:
data['categories']

In [0]:
for i in data['images']:
  if i['id'] == 289343:
    img = Image('/content/gdrive/My Drive/cocostuff2017/val2017/'+i['file_name'])
    display(img)

In [0]:
data['annotations'][0]

In [0]:
for a in data['annotations']:
  if a['image_id']==289343:
    print (a['category_id'])

In [0]:
data['annotations'][0]

In [0]:
data['annotations'][0]